# Earthquake Data Analysis

### Description

The catalog includes the magnitude, time of occurrence (s), and 3D coordinates (m) of earthquakes in about 20 years of recording in South California. Coordinates were converted from latitude, longitude, and depth of events in a seismic catalog. Magnitudes should be within the range $[0,8]$.

* **Waiting time (t)**: time interval between an event and the next one in the sequence.
* **Distance (r)**: Eucledian 3D distance between events. (each 3D set of coordinates refers to the hypocenter, i.e. the point triggering the slip in a fault that forms the earthquake)


### Assignments

1. Deduce what is the variable in each column of the catalog.
2. Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.
3. Compute the distribution $P_m(t)$ of waiting times for events of magnitude m or above (i.e. do not consider events below $m$). In shaping the bin sizes, take into account that this distribution is expected to have a power-law decay with time (e.g $\sim 1/t$), and that a power-law is well visualized in log-log scale. Do this analysis for many values of $m$, say $m=2,3,4,5$.
4. Compute the distribution $P_m(r)$ of the distance between an event and the next one, considering earthquakes of magnitude m or above. Also here make a clever choice for the bin sizes and try several values of $m$.
5. Compute the distribution $P_{m,R}(t)$ of waiting times for events of magnitude $m$ or above, which are separated by at most a distance $r<R$, for different values of m and $R$. (In this statistics, if the following event is farther than $R$, skip the $t$ and go to the next pair)
6. Eventually note if, from the analysis of the previous points, there emerges a scaling picture. Is there a suitable rescaling that collapses distributions for various $m$ (and eventually $R$ if point 5 is considered) on a single curve?

### Datasets

* column 1: index of the event
* column 2: index of the previous event that triggered it (defined with a given algorithm), -1 if no ancestor is found
* column 3: time (seconds) from 0:00 of Jan.1st, 1982
* column 4: magnitude
* columns 5, 6, and 7: 3D coordinates (meters) of the earthquake hypocenter, i.e. of the point from where it started. These Euclidean coordinates are derived from latitude, longitude and depth.

Joining each event to that with the index of the second column (if not -1), there emerges a set of causal trees.


### Contact
* Marco Baiesi <marco.baiesi@unipd.it>

In [20]:
from pandas import DataFrame
import pandas as pd
import numpy as np 
import math 
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats

import ipywidgets as widgets
from ipywidgets import interactive

In [21]:
# creazione dataframe

filename = "SouthCalifornia-1982-2011_Physics-of-Data.dat"
labels = ("pointer","t","mag","x","y","z")

d=pd.read_csv(filename,sep="\s",names=labels,engine='python')
df = pd.DataFrame(data=d)
df

,pointer,t,mag,x,y,z
0,-1,0.000000e+00,2.71,-2571956,-4627162,3520602
1,0,3.650139e+04,2.12,-2363740,-4787011,3461373
2,0,3.748828e+04,2.33,-2363746,-4786942,3461232
3,0,4.798252e+04,2.57,-2475085,-4664024,3548479
4,0,6.026857e+04,2.98,-2238642,-4839098,3469546
...,...,...,...,...,...,...
110266,-1,9.304996e+08,2.60,-2668492,-4335735,3810743
110267,-1,9.305115e+08,2.02,-2297480,-4823870,3445285
110268,-1,9.305318e+08,2.00,-2404797,-4441247,3868121
110269,-1,9.305363e+08,2.17,-2388375,-4691191,3550903


In [46]:
def chi_square (obs, exp, error):
    chisqr = 0
    size = len(obs)
    for i in range(size):
        chisqr = chisqr + ((obs[i]-exp[i])/(error[i]))**2
    return chisqr

In [70]:
#le due pdf usate per modellizzare P(r), con r appartenente a [1,650]
#600 km e'una soglia imposta a priori, ed e' dovuta al fatto che il dataset e' ristretto
#a una zona limitata, e quindi r non puo' assumere valori arbitrariamente grandi. Per valori > 650 km infatti
#la distribuzione cala rapidamente, come si vedra' negli histo con log log scale

#modello base di fit, quello che dovrebbe essere piu corretto
def fit_model (r,p0,p1):
    return p0*(r)**(-p1)

# altro modello giusto per provare
#abs() e' per porre p0 > 0, altrimenti non regge la defizione di pdf
#def fit_pdf2 (r,p0):
#    p0 = abs(p0)
#    p1 = 1./(math.log( (650+p0)/p0) ) 
#   return p1/(r+p0)


########## funzione per ottenere e analizzare gli istogrammi di P(r) #########
def r_distr (m,n_bins,color,edgecolor,x_min_fit):
# df = dataframe 
# m = soglia di magnitudo
# n_bins = numero bin histo
# x_min_fit = valore delle x per cui iniziano a vedersi le curve dei fit nel grafico senza logscale,
# e' modificabile in modo da permettere sempre una buona visualizzazione visto che per r->0 fit_1 diverge

    #il range degli histo e' [0,km_range]
    km_range = 1000

    #dataframe filtrato per certi valori della magnitudo
    dr = df[df["mag"]>m]
    
    #uso le colonne del dataframe per avere le distanze con cui fare l'istogramma
    distance = []
    X = np.array(dr["x"].tolist())
    Y = np.array(dr["y"].tolist())
    Z = np.array(dr["z"].tolist())
    n_events = len(X)
    
    #distanza calcolata in km
    for i in range(0,n_events-1):
        d = np.sqrt( (X[i]-X[i+1])**2. + (Y[i]-Y[i+1])**2. + (Z[i]-Z[i+1])**2. )
        distance.append(d/1000)   

######## creazione degli histo relativi, uno con scale normali e l'altro con doppia log scale ########
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    
    y_bins, x_bins, p = ax1.hist(distance,bins=n_bins,range=(0,km_range),
                                 color=color, density=True, edgecolor=edgecolor,label="Distance distribution")
    
    #errore poissoniano dei bins, calcolato tenendo conto del fatto che y e' normalizzato. Usato per il fit
    y_bins_err = []
    for yi in y_bins: y_bins_err.append( np.sqrt(yi/n_events) )
    
    title = "r distribution for magnitude above m=" + str(m)
    #ax1.grid()
    ax1.set_title(title)
    ax1.set_ylabel("Pm(r)")
    ax1.set_xlabel("r (km)")
    
    bins = np.logspace(0,3,n_bins)
    ax2.hist(distance,bins=bins,range=(0,km_range),color=color,edgecolor=edgecolor,density=True,label="Distance distribution")
    title = "log scales r distribution for magnitude above m=" + str(m)
    #ax2.grid()
    ax2.set_title(title)
    ax2.set_ylabel("Pm(r)")
    ax2.set_xlabel("r (km)")
    ax2.set_yscale('log') 
    ax2.set_xscale('log')
    
########### fit dell'histo con le scale normali ##########

    #si centrano i bins e si leva bin in eccesso
    x_bins = x_bins+km_range/(2*n_bins)
    x_bins = np.delete(x_bins,n_bins)
    
    #si considerano solo bin all'interno del "range del fit" e i bin che che hanno contenuto diverso da 0
    index = []
    for i in range(n_bins):
        if(x_bins[i]>650 or y_bins_err[i]==0):  index.append(i)
       
    x = np.delete(x_bins,index)
    y = np.delete(y_bins,index)
    y_err = np.delete(y_bins_err,index)
    
    #questi linspace servono a visualizzare i fit, il numero 1 e' quello del grafico senza logscale,
    #proprio per questo motivo non la faccio partire da 1 altrimenti l'histo blu nemmeno si vedrebbe da quanto 
    #sale il modello
    xspace1 = np.linspace(x_min_fit, 650, 100000)
    xspace2 = np.linspace(1, 650, 100000)
    
    #fit del modello 
    par1, pcov1 = curve_fit(fit_model, xdata=x, ydata=y, sigma=y_err, p0=[0.15,0.9])
    ax1.plot(xspace1, fit_model(xspace1, *par1), 'black', linewidth=2., label = "First model")
    ax2.plot(xspace2, fit_model(xspace2, *par1), 'black', linewidth=2., label = "First model")
    
    p0 = par1[0]
    p1 = par1[1]
    print("Model:   Pm(r) = K*r^-p")
    print("K =", p0, "+-", np.sqrt(pcov1[0][0]) )
    print("p =", p1, "+-", np.sqrt(pcov1[1][1]) )
    print("Chi-squared:", chi_square(fit_model(x,p0,p1),y,y_err), "/" , len(y)-2)
        
    ax1.legend()
    ax2.legend()
    return 

In [71]:
interactive_plot = interactive(r_distr, m=(2, 5, 0.5), n_bins=(5,75,5), 
                               color=['lightblue', 'red', 'blue', 'cyan', 'orange', 'yellow', 'green', 'black'],
                              edgecolor=['blue', 'lightblue', 'red', 'cyan', 'orange', 'yellow', 'green', 'black'],
                              x_min_fit =(3,23,5))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

#la coda degli istogrammi cala repentinamente a causa del fatto che il dataset e' ristretto
#a una zona limitata, e quindi r non puo' assumere valori arbitrariamente grandi.

interactive(children=(FloatSlider(value=3.0, description='m', max=5.0, min=2.0, step=0.5), IntSlider(value=40,…

Result for model with n_bin = 40

| m | K | p | chi | v 
| :---: | :---: | :---: | :---: | :---: 
| 2 | 0.11 $\pm$ 0.02 | 0.88 $\pm$ 0.04 | 280 | 24 
| 2.5 | 0.11 $\pm$ 0.02 | 0.88 $\pm$ 0.04 | 82 | 24 
| 3 | 0.12 $\pm$ 0.02 | 0.92 $\pm$ 0.04 | 27 | 24 
| 3.5 | 0.14 $\pm$ 0.03 |  0.94 $\pm$ 0.04 | 9 | 24 
| 4 | 0.14 $\pm$ 0.03 | 0.95 $\pm$ 0.04 | 3 | 24 
| 4.5 | 0.19 $\pm$ 0.04 | 1.03 $\pm$ 0.05 | 1.4 | 24 
| 5 | 0.11 $\pm$ 0.03 | 0.90 $\pm$ 0.07 | 0.5 | 20 


In [72]:
#all'aumentare di m gli eventi sono sempre meno e l'errore poissoniano pesa sempre di piu'
#per m compreso tra 2 e 2.5 c'e concentrazione di eventi che avvengono a distanza di 200 km